Jupyter notebook to distignish Cats and Dogs

In [62]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

Global vars

In [63]:
TRAIN_DIR = 'train'
TEST_DIR = 'test'
IMG_SIZE = 50
LR = 1e-3

print("done loading")

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match

done loading


Create training labels

In [64]:
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]

In [65]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        if img[-4:] == ".jpg":
            label = label_img(img)
            path = os.path.join(TRAIN_DIR,img)
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

Preprocess test data

In [66]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    print(saved)
    return testing_data

Create training data

In [19]:
#train_data = create_train_data()

100%|██████████| 25001/25001 [00:37<00:00, 672.15it/s]


Loading trainig data

In [67]:
train_data = np.load('train_data.npy')

Setup tensorflow

In [68]:
import tensorflow as tf

In [69]:
def neural_net_image_input(image_shape):
    return tf.placeholder(tf.float32, shape=[None, image_shape[0], image_shape[1], image_shape[2]], name="x")

In [70]:
def neural_net_label_input(n_classes):
    return tf.placeholder(tf.float32, shape=[None, n_classes], name="y")

In [71]:
def neural_net_keep_prob_input():
    return tf.placeholder(tf.float32, name="keep_prob")

In [72]:
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    depth = int(x_tensor.shape[3])
    filter_weights = tf.Variable(tf.truncated_normal([conv_ksize[0], conv_ksize[1],depth, conv_num_outputs], stddev=0.1))
    filter_bias = tf.Variable(tf.zeros(conv_num_outputs))
    stridesTuple = [1, conv_strides[0], conv_strides[1], 1] # (batch, height, width, depth)
    paddingString = 'SAME'
    conv = tf.nn.conv2d(x_tensor, filter_weights, strides=stridesTuple, padding=paddingString)
    conv = tf.nn.bias_add(conv, filter_bias)
    conv = tf.nn.relu(conv)
    poolTuple = [1, pool_strides[0], pool_strides[1], 1] # (batch, height, width, depth)
    conv = tf.nn.max_pool(conv,ksize=[1, pool_ksize[0], pool_ksize[1], 1],strides=poolTuple,padding=paddingString)
    return conv

In [73]:
def flatten(x_tensor):
    return tf.contrib.layers.flatten(x_tensor)

In [29]:
def fully_conn(x_tensor, num_outputs):
    return tf.contrib.layers.fully_connected(inputs=x_tensor, num_outputs=num_outputs, activation_fn=tf.nn.relu)

In [30]:
def output(x_tensor, num_outputs):
    return tf.contrib.layers.fully_connected(inputs=x_tensor, num_outputs=num_outputs, activation_fn=None)

In [31]:
def conv_net(x, keep_prob):
    conv = conv2d_maxpool(x, 16, (8,8), (1,1), (2,2), (2,2))
    conv = conv2d_maxpool(x, 32, (4,4), (1,1), (2,2), (2,2))
    conv = conv2d_maxpool(x, 64, (2,2), (1,1), (2,2), (2,2))

    conv = flatten(conv)
    
    conv = fully_conn(conv, 200)
    conv = tf.nn.dropout(conv, keep_prob)
    conv = fully_conn(conv, 100)
    conv = tf.nn.dropout(conv, keep_prob)
    

    conv = output(conv, 2)
    
    return conv

In [32]:
# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = neural_net_image_input((IMG_SIZE, IMG_SIZE, 1))
y = neural_net_label_input(2)
keep_prob = neural_net_keep_prob_input()

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [33]:
def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    output = session.run(optimizer, feed_dict={x: feature_batch, y: label_batch, keep_prob: keep_probability})
    pass


In [56]:
epochs = 10
batch_size = 10
keep_probability = 0.7

Training

In [57]:
train = train_data[:-500]
test = train_data[-500:]

In [58]:
train_x = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
train_y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [74]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    loss = session.run(cost, feed_dict={x: feature_batch, y: label_batch, keep_prob: 1.0})
    valid_acc = session.run(accuracy, feed_dict={x: test_x, y: test_y, keep_prob: 1.0})
    print('Loss: {} Validation Accuracy: {}'.format(loss, valid_acc))
    pass

Split data in batches

In [75]:
def chunks(x, y, n):
    for i in range(0, len(x), n):
        yield x[i:i + n], y[i:i + n] 

In [76]:
batches = chunks(train_x, train_y, batch_size)

In [ ]:
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        batch_i = 0
        for batch_x, batch_y in batches:
            batch_i += 1
            train_neural_network(sess, optimizer, keep_probability, batch_x, batch_y)
            print('Epoch {:>2}, CatsAndDogs Batch {}:  '.format(epoch + 1, batch_i), end='')
            print_stats(sess, batch_x, batch_y, cost, accuracy)
            
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

Epoch  1, CatsAndDogs Batch 1:  Loss: 32.89157485961914 Validation Accuracy: 0.5180000066757202
Epoch  1, CatsAndDogs Batch 2:  Loss: 43.42072677612305 Validation Accuracy: 0.5099999904632568
Epoch  1, CatsAndDogs Batch 3:  Loss: 192.65512084960938 Validation Accuracy: 0.49000000953674316
Epoch  1, CatsAndDogs Batch 4:  Loss: 175.56063842773438 Validation Accuracy: 0.49000000953674316
Epoch  1, CatsAndDogs Batch 5:  Loss: 75.60923767089844 Validation Accuracy: 0.4880000054836273
Epoch  1, CatsAndDogs Batch 6:  Loss: 151.8937225341797 Validation Accuracy: 0.5099999904632568
Epoch  1, CatsAndDogs Batch 7:  Loss: 298.9695739746094 Validation Accuracy: 0.5099999904632568
Epoch  1, CatsAndDogs Batch 8:  Loss: 335.15484619140625 Validation Accuracy: 0.5099999904632568
Epoch  1, CatsAndDogs Batch 9:  Loss: 523.6015625 Validation Accuracy: 0.5099999904632568
Epoch  1, CatsAndDogs Batch 10:  Loss: 222.1463623046875 Validation Accuracy: 0.5099999904632568
Epoch  1, CatsAndDogs Batch 11:  Loss: 1